<a href="https://colab.research.google.com/github/swaminaathakrishnan/Cool_Route_prototype/blob/master/Cool_route_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚴 **CoolRide V3: AI-Driven Thermal Routing System**
### Project Overview

CoolRide is a TRL-6 prototype designed to route cyclists through thermally comfortable paths in Singapore. It utilizes real-time government weather data (NEA), satellite imagery (Vegetation), and predictive AI to mitigate Urban Heat Island (UHI) effects.

In [ ]:
# ==========================================
# 🧱 MODULE 1: SYSTEM INITIALIZATION
# ==========================================
# Objective: Install geospatial libraries and set up the environment.
# dependencies: OSMnx (Maps), GeoPandas (Spatial Data), Scikit-Learn (AI).

!pip install osmnx simplekml geopandas shapely networkx requests scikit-learn -q

import osmnx as ox
import networkx as nx
import simplekml
import geopandas as gpd
import pandas as pd
import requests
import numpy as np
import os
import pickle
import time
from shapely.geometry import Point, LineString, box
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

print("✅ System Initialized. Ready for V3 Execution.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.0 MB/s eta 0:00:00
✅ System Initialized. Ready for V3 Execution.


### ⚙️ Module 2: Configuration & Cloud Connection
Objective: Define the pilot zone and connect to the GitHub Data Lake. Logic: Instead of local files, we stream GeoJSON/CSV directly from the raw GitHub URLs. This allows the team to collaborate without sharing Drive folders.

In [ ]:
# ==========================================
# ⚙️ MODULE 2: CONFIGURATION
# ==========================================

# 1. PILOT ZONE
PLACE_NAME = "Tampines, Singapore"
START_POINT = (1.3533, 103.9452) # Tampines MRT
END_POINT = (1.3598, 103.9351)   # Tampines Eco Green

# 2. GITHUB DATA LAKE (Edit these to match your repo!)
GITHUB_USER = "swaminaathakrishnan"
REPO_NAME = "Cool_Route_prototype"
BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/master/data/"

# File Links
PCN_URL = BASE_URL + "ParkConnectorLoop.geojson"
HAWKER_URL = BASE_URL + "HawkerCentresGEOJSON.geojson"
TREES_URL = BASE_URL + "trees.csv" # The new SGTrees Data

# 3. SAFETY OVERRIDE (When Govt. advisory is issued)
NEA_HEATWAVE_ALERT = False # Set True to force extreme caution

print(f"✅ Configuration Loaded. Connecting to Data Lake at: {GITHUB_USER}/{REPO_NAME}")

✅ Configuration Loaded. Connecting to Data Lake at: swaminaathakrishnan/Cool_Route_prototype


### 🗺️ Module 3: The Spatial Graph Engine (With SGTrees)

Objective: Build the road network and overlay specific cooling features. _Upgrade: Now includes Individual Tree Analysis._

Layer 1: Road Network (OSM).

Layer 2: Park Connectors (PCN).

Layer 3: Individual Trees (SGTrees). Logic: Roads with high tree density receive a "Shade Bonus," lowering their travel cost significantly.

In [ ]:
# ==========================================
# 🗺️ MODULE 3: SPATIAL GRAPH ENGINE (V3.1 - LFS FIXED)
# ==========================================
import os

def generate_cool_routes():
    print(f"⏳ Downloading road network for {PLACE_NAME}...")

    # 1. GET GRAPH
    G = ox.graph_from_point(START_POINT, dist=2000, network_type='bike')
    nodes = ox.graph_to_gdfs(G, edges=False)
    miny, maxy = nodes.y.min(), nodes.y.max()
    minx, maxx = nodes.x.min(), nodes.x.max()
    print(f"   📐 Zone Limits: Lat[{miny:.4f}, {maxy:.4f}], Lon[{minx:.4f}, {maxx:.4f}]")

    # 2. LOAD PCN DATA
    print("⏳ Overlaying Park Connectors...")
    try:
        pcn_data = gpd.read_file(PCN_URL)
        if pcn_data.crs != "EPSG:4326": pcn_data = pcn_data.to_crs("EPSG:4326")
        # FIX: Updated deprecated unary_union to union_all() if available, else unary_union
        try: pcn_union = pcn_data.geometry.union_all()
        except: pcn_union = pcn_data.geometry.unary_union
    except:
        print("   ⚠️ PCN Data missing. Proceeding without it.")
        pcn_union = None

    # 3. LOAD SGTREES DATA (LFS FIX)
    print("⏳ Loading Tree Data (Force Download)...")
    trees_buffer = None
    local_tree_file = "trees_downloaded.csv"

    try:
        # A. Force Download using WGET (Handles LFS redirects better than Pandas)
        # We use the 'raw' URL structure which usually triggers the LFS blob download
        lfs_url = f"https://github.com/{GITHUB_USER}/{REPO_NAME}/raw/master/data/trees.csv"
        os.system(f"wget -O {local_tree_file} {lfs_url}")

        # B. Read the local file
        trees_df = pd.read_csv(local_tree_file)

        # C. Check if we still got the pointer (The "4KB trap")
        if len(trees_df) < 50 and "version https" in str(trees_df.iloc[0]):
            raise ValueError("LFS Pointer detected! The 48MB file did not download.")

        # D. Normalize Columns
        cols = [c.lower() for c in trees_df.columns]
        trees_df.columns = cols
        lat_col = 'latitude' if 'latitude' in cols else 'lat'
        lng_col = 'longitude' if 'longitude' in cols else 'lng'

        # E. Bounding Box Filter
        trees_df = trees_df[
            (trees_df[lat_col] >= miny) & (trees_df[lat_col] <= maxy) &
            (trees_df[lng_col] >= minx) & (trees_df[lng_col] <= maxx)
        ]

        print(f"   ✂️ Filtered Trees: Keeping {len(trees_df)} trees for {PLACE_NAME}.")

        if len(trees_df) > 0:
            geometry = [Point(xy) for xy in zip(trees_df[lng_col], trees_df[lat_col])]
            trees_gdf = gpd.GeoDataFrame(trees_df, geometry=geometry, crs="EPSG:4326")
            trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
            print(f"   ✅ Shade Layer Generated.")

    except Exception as e:
        print(f"   ⚠️ Tree Data Error: {e}. Skipping micro-shade analysis.")

    # 4. LOAD SHELTERS
    shelters = []
    try:
        hawker_data = gpd.read_file(HAWKER_URL)
        hawker_data = hawker_data.cx[minx:maxx, miny:maxy]
        for _, row in hawker_data.iterrows():
            name = row.get('Name') or row.get('NAME') or 'Shelter'
            shelters.append((name, row.geometry.y, row.geometry.x))
    except: pass

    # 5. CALCULATE COST
    print("⏳ Calculating 'Micro-Shade' Scores...")
    for u, v, k, data in G.edges(keys=True, data=True):
        if 'geometry' in data:
            edge_geom = data['geometry']
        else:
            edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])])

        cost = data['length']
        is_pcn = False
        if pcn_union and edge_geom.intersects(pcn_union):
            is_pcn = True
            cost *= 0.5

        has_shade = False
        if trees_buffer and edge_geom.intersects(trees_buffer):
            has_shade = True
            cost *= 0.6

        data['cool_cost'] = cost
        data['tag'] = "🌿 PCN + 🌳 Canopy" if (is_pcn and has_shade) else "🌿 PCN" if is_pcn else "🌳 Shaded Road" if has_shade else "☀️ Exposed"

    # 6. SOLVE
    orig = ox.distance.nearest_nodes(G, START_POINT[1], START_POINT[0])
    dest = ox.distance.nearest_nodes(G, END_POINT[1], END_POINT[0])

    try:
        r_fast = nx.shortest_path(G, orig, dest, weight='length')
        r_cool = nx.shortest_path(G, orig, dest, weight='cool_cost')
        return G, r_fast, r_cool, shelters
    except:
        return None, None, None, None

### 🧠 Module 4: The Historical AI Engine (Self-Healing)

Objective: Predict short-term WBGT trends using historical data. Features:

Pagination Logic: Fetches huge datasets by turning API pages.

Physics Clamping: Prevents unrealistic temperature predictions (>0.5°C swings).

Self-Healing Cache: Automatically deletes bad cache files and re-learns.

In [ ]:
# ==========================================
# 🧠 MODULE 4: HISTORICAL AI ENGINE (V3)
# ==========================================
CACHE_FILE = "coolride_weather_memory.pkl"

def get_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "rb") as f: return pickle.load(f)
        except: return {}
    return {}

def save_cache(data):
    with open(CACHE_FILE, "wb") as f: pickle.dump(data, f)

def fetch_historical_data(station_name, days_back=3):
    cache = get_cache()
    today_str = datetime.now().strftime("%Y-%m-%d")
    cache_key = f"{station_name}_{today_str}"

    if cache_key in cache and len(cache[cache_key]['values']) > 20:
        print(f"   ⚡ Memory Hit! Loaded {len(cache[cache_key]['values'])} points.")
        return cache[cache_key]['timestamps'], cache[cache_key]['values']

    print(f"   📡 Memory Miss. Analyzing last {days_back} days...")
    all_timestamps, all_values = [], []

    for i in range(days_back + 1):
        target_date = (datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d")
        url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
        params = {"api": "wbgt", "date": target_date}

        # PAGINATION LOOP
        while True:
            try:
                resp = requests.get(url, params=params, timeout=5)
                if resp.status_code != 200: break
                data = resp.json()
                if 'data' not in data: break

                for rec in data['data'].get('records', []):
                    dt = datetime.fromisoformat(rec['datetime'])
                    for r in rec['item']['readings']:
                        if r.get('station', {}).get('name') == station_name:
                            val = float(r.get('wbgt') or r.get('value'))
                            mins = dt.hour * 60 + dt.minute
                            now_mins = datetime.now().hour * 60 + datetime.now().minute
                            if abs(mins - now_mins) < 240: # 4 hour window
                                all_timestamps.append(mins)
                                all_values.append(val)

                token = data['data'].get('paginationToken')
                if token: params['paginationToken'] = token
                else: break
            except: break

    if len(all_values) > 20:
        cache[cache_key] = {'timestamps': all_timestamps, 'values': all_values}
        save_cache(cache)
        print(f"   💾 Learned & Saved {len(all_values)} thermal patterns.")

    return all_timestamps, all_values

def predict_trend(station_name, current_wbgt):
    timestamps, values = fetch_historical_data(station_name)
    if len(values) < 10: return current_wbgt, "Stable ➖", "Low Data"

    # Linear Regression
    model = LinearRegression()
    model.fit(np.array(timestamps).reshape(-1, 1), np.array(values))

    # Forecast
    now = datetime.now()
    fut_min = now.hour * 60 + now.minute + 15
    raw_pred = model.predict([[fut_min]])[0]

    # Physics Clamp
    delta = raw_pred - current_wbgt
    if abs(delta) > 0.5:
        final_pred = current_wbgt + (0.5 if delta > 0 else -0.5)
        note = "(Physics Clamped)"
    else:
        final_pred = raw_pred
        note = ""

    trend = "Rising 📈" if final_pred > current_wbgt + 0.1 else "Falling 📉" if final_pred < current_wbgt - 0.1 else "Stable ➖"
    return final_pred, trend, f"High {note}"

### 🚀 Module 5: Execution & Safe-Pace Recommendations

Objective: Synthesize map, weather, and AI data into a KML route. Upgrade:

* Govt Override: Checks NEA_HEATWAVE_ALERT.
* Safe Pacing: Suggests specific ride speeds and hydration intervals based on WBGT (ISO 7243 standards).

In [ ]:
# ==========================================
# 🚀 MODULE 5: EXECUTION ENGINE (V3.9 - FINAL LABELS)
# ==========================================
import math
import os
import time
import requests
import simplekml

print("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")

# --- HELPER: DYNAMIC SENSOR FINDER (FULL V2 LOGIC) ---
def get_nearest_wbgt_station(lat, lon):
    print("⏳ Connecting to NEA Official WBGT Sensor Network...")
    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
    try:
        resp = requests.get(url, params={"api": "wbgt"}, timeout=10)
        data = resp.json()
        readings = data['data']['records'][0]['item'].get('readings', [])

        closest_station = "Unknown"
        min_dist = float('inf')
        current_val = None

        for r in readings:
            try:
                loc = {}
                s_name = "Unknown"
                if 'location' in r: loc = r['location']
                elif 'station' in r and 'location' in r['station']: loc = r['station']['location']
                if 'station' in r: s_name = r['station'].get('name', 'Unknown')
                s_lat = float(loc.get('latitude', 0))
                s_lon = float(loc.get('longitude', loc.get('longtitude', 0)))
                if s_lat == 0 or s_lon == 0: continue

                val = r.get('wbgt') or r.get('value')
                if val is None: continue
                val = float(val)
                dist = math.sqrt((lat - s_lat)**2 + (lon - s_lon)**2)

                if dist < min_dist:
                    min_dist = dist
                    closest_station = s_name
                    current_val = val
            except: continue

        if current_val is None: return 30.0, "System Fallback"
        print(f"   📍 Nearest Sensor: {closest_station} (Dist: {min_dist*111:.2f} km)")
        return current_val, closest_station
    except Exception as e:
        print(f"   ⚠️ WBGT Sensor Fail: {e}. Using Default Safety Value.")
        return 30.0, "System Fallback"

# 1. GENERATE ROUTES
graph, r1, r2, shelters = generate_cool_routes()

if graph:
    # 2. GET WEATHER
    current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    # 3. RUN AI
    pred_wbgt, trend, confidence = predict_trend(station_name, current_wbgt)
    effective_wbgt = max(current_wbgt, pred_wbgt)
    if NEA_HEATWAVE_ALERT: effective_wbgt = 35.0

    # 4. REPORT
    if effective_wbgt < 29: rec = "✅ Safe to Ride."
    elif effective_wbgt < 31: rec = "⚠️ CAUTION: Seek shade."
    else: rec = "🛑 HIGH RISK: Stop."

    print(f"\n📊 REPORT: {station_name}")
    print(f"   Current: {current_wbgt}°C | Forecast: {pred_wbgt:.1f}°C")

    # 5. EXPORT KML (CLEAN LABELS)
    kml = simplekml.Kml()

    def add_route(route, color, name, description):
        ls = kml.newlinestring(name=name)
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            d = graph.get_edge_data(u, v)[0]
            if 'geometry' in d:
                xs, ys = d['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        ls.coords = coords
        ls.style.linestyle.color = color
        ls.style.linestyle.width = 5
        ls.description = description

    # 🧠 FUZZY LOGIC + CLEAN LABELS
    def check_similarity(route_a, route_b):
        set_a = set(route_a)
        set_b = set(route_b)
        intersection = len(set_a.intersection(set_b))
        union = len(set_a.union(set_b))
        return intersection / union

    sim_score = check_similarity(r1, r2)
    print(f"   🔍 Route Similarity Score: {sim_score*100:.1f}%")

    if sim_score > 0.90:  # If >90% similar, merge them
        print("   💡 Insight: Routes are effectively identical (Merged).")
        # MERGED LABEL
        add_route(r2, simplekml.Color.green, "🌟 Recommended Route",
                  f"<b>Smart Choice</b><br>The fastest path is also the coolest.<br>No detour needed.<br><br>Temp: {effective_wbgt:.1f}°C")
    else:
        print("   💡 Insight: A distinct cooler detour exists.")
        # DIVERGENT LABELS
        add_route(r1, simplekml.Color.red, "⚡ Fastest Route (Exposed)",
                  f"<b>Direct Path</b><br>Shortest time, but higher heat exposure.<br><br>Temp: {effective_wbgt:.1f}°C")
        add_route(r2, simplekml.Color.green, "🌿 Cool Route (Shaded)",
                  f"<b>Shaded Detour</b><br>Maximized tree canopy coverage.<br>Lower heat stress.<br><br>Temp: {effective_wbgt:.1f}°C")

    # Add Shelters
    if shelters:
        for name, lat, lon in shelters:
            p = kml.newpoint(name=f"🧊 {name}", coords=[(lon, lat)])
            p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/snowflake_simple.png'

    # 6. SAVE
    if not os.path.exists('output'): os.makedirs('output')
    constant_filename = "output/latest_route.kml"
    kml.save(constant_filename)

    print(f"\n🎉 SUCCESS! Download '{constant_filename}'")
else:
    print("❌ Critical Error: Route Generation Failed.")

🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...
⏳ Downloading road network for Tampines, Singapore...
   📐 Zone Limits: Lat[1.3353, 1.3713], Lon[103.9273, 103.9632]
⏳ Overlaying Park Connectors...
⏳ Loading Tree Data (Force Download)...
   ✂️ Filtered Trees: Keeping 29766 trees for Tampines, Singapore.


/tmp/ipython-input-2707566625.py:63: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
/tmp/ipython-input-2707566625.py:63: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union


   ✅ Shade Layer Generated.
⏳ Calculating 'Micro-Shade' Scores...
⏳ Connecting to NEA Official WBGT Sensor Network...
   📍 Nearest Sensor: Bedok North Street 2 (Dist: 3.12 km)
   ⚡ Memory Hit! Loaded 128 points.

📊 REPORT: Bedok North Street 2
   Current: 27.3°C | Forecast: 26.8°C
   🔍 Route Similarity Score: 93.1%
   💡 Insight: Routes are effectively identical (Merged).

🎉 SUCCESS! Download 'output/latest_route.kml'
